# Document-level Relation Extraction Tutorial

> Tutorial author: 黎洲波（zhoubo.li@zju.edu.cn）

In this tutorial, we use [DocuNet](http://arxiv.org/abs/2106.03618) to extract relational triples in different sentences. We hope this tutorial can help you understand the process of document-level relation extraction.

This tutorial uses `Python3`.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sat Apr 22 08:44:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## RE
**Relation extraction** (RE), a key task in information extraction, predicts semantic relations between pairs of entities from unstructured
texts.

## Document-level RE
Document-level RE extracts relations from multi-sentence in one document. An example is shown in the following picture, in which named entities are annotated with colors. Different from sentence-level RE, document-level RE can extract both intra-sentence and inter-sentence relational triples.
![文档级关系抽取](https://github.com/zjunlp/DeepKE/blob/main/tutorial-notebooks/re/document/img/img1.png?raw=true)

## Dataset

There are some document-level RE datasets including DocRED, CDR and GDA. The tutorial uses [DocRED](https://github.com/thunlp/DocRED/tree/master/). The structure of the dataset folder `./data/` is as follow:

```
.
├── dev.json                        # Validation Set
├── rel_info.json                   # Relation Label
├── rel2id.json                     # Relation Label - ID Map
├── test.json                       # Test Set
└── train_annotated.json            # Training Set
```

The data formats of DocRED are described as follow:

```
Data Format:
{
  'title',
  'sents':     [
                  [word in sent 0],
                  [word in sent 1]
               ]
  'vertexSet': [
                  [
                    { 'name': mention_name,
                      'sent_id': mention in which sentence,
                      'pos': postion of mention in a sentence,
                      'type': NER_type}
                    {anthor mention}
                  ],
                  [anthoer entity]
                ]
  'labels':   [
                {
                  'h': idx of head entity in vertexSet,
                  't': idx of tail entity in vertexSet,
                  'r': relation,
                  'evidence': evidence sentences' id
                }
              ]
}
```

## DocuNet
- [DocuNet](http://arxiv.org/abs/2106.03618) used in DeepKE is a semantic segmentation method using Document U-shaped Network based on computer vision (CV) and obtains excellent performance on DocRED dataset.
- The framework of DocuNet is as follow:

![文档级关系抽取架构图](https://github.com/zjunlp/DeepKE/blob/main/tutorial-notebooks/re/document/img/img2.png?raw=true)

## Prepare the runtime environment

In [ ]:
# !pip install deepke
# !wget 120.27.214.45/Data/re/document/data.tar.gz
# !tar -xzvf data.tar.gz

In [ ]:
# ! cd /content/drive/MyDrive/Models; git clone https://github.com/zjunlp/DeepKE

Cloning into 'DeepKE'...
remote: Enumerating objects: 8949, done.
remote: Counting objects: 100% (743/743), done.
remote: Compressing objects: 100% (434/434), done.
remote: Total 8949 (delta 396), reused 594 (delta 294), pack-reused 8206
Receiving objects: 100% (8949/8949), 107.52 MiB | 16.87 MiB/s, done.
Resolving deltas: 100% (4510/4510), done.
Updating files: 100% (570/570), done.


In [ ]:
# !pip install -r /content/drive/MyDrive/Models/DeepKE/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 750.6/750.6 MB 200.0 MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [ ]:
!pip install -U torch ujson transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.5 MB/s eta 0:00:00


## Import modules

In [ ]:
import os
import time
import numpy as np
import torch
import random
import pickle
from tqdm import tqdm
import ujson as json
from opt_einsum import contract

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import AutoConfig, AutoModel, AutoTokenizer
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

from tqdm import tqdm

In [ ]:
# os.chdir('../')

In [ ]:
!pwd

/content


## Preprocess the dataset

In [ ]:
rel2id = json.load(open('/content/drive/MyDrive/data/nerel_docred_ver/rel2id.json', 'r'))
#rel2id = json.load(open('/Users/yandutov-alex/study/science_work/DocRED_original/code/prepro_data/rel2id.json'))

id2rel = {value: key for key, value in rel2id.items()}


def chunks(l, n):
    res = []
    for i in range(0, len(l), n):
        assert len(l[i:i + n]) == n
        res += [l[i:i + n]]
    return res

class ReadDataset:
    def __init__(self, dataset: str, tokenizer, max_seq_Length: int = 1024,
             transformers: str = 'bert') -> None:
        self.transformers = transformers
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_seq_Length = max_seq_Length

    def read(self, file_in: str):
        save_file = file_in.split('.json')[0] + '_' + self.transformers + '_' \
                        + self.dataset + '.pkl'
        if self.dataset == 'docred':
            return read_docred(self.transformers, file_in, save_file, self.tokenizer, self.max_seq_Length, ents='nerel')
        # elif self.dataset == 'nerel':
        #     return read_docred(self.transformers, file_in, save_file, self.tokenizer, self.max_seq_Length, ents='nerel')
        else:
            raise RuntimeError("No read func for this dataset.")

def read_docred(transfermers, file_in, save_file, tokenizer, max_seq_length=1024, ents='docred'):
    if os.path.exists(save_file):
        with open(file=save_file, mode='rb') as fr:
            features = pickle.load(fr)
            fr.close()
        print('load preprocessed data from {}.'.format(save_file))
        return features
    else:
        max_len = 0
        up512_num = 0
        i_line = 0
        pos_samples = 0
        neg_samples = 0
        features = []
        if file_in == "":
            return None
        with open(file_in, "r") as fh:
            data = json.load(fh)

        if transfermers == 'bert':
            # entity_type = ["ORG", "-",  "LOC", "-",  "TIME", "-",  "PER", "-", "MISC", "-", "NUM"]
            if ents == 'nerel':
                entity_type = ['-', 'IDEOLOGY', '-', 'PENALTY', '-', 'CITY', '-', 'CRIME', '-', 'DISEASE', '-', 'TIME', '-', 'PERSON', '-',
                 'RELIGION', '-', 'EVENT', '-', 'AWARD', '-', 'LANGUAGE', '-', 'ORDINAL', '-', 'ORGANIZATION', '-', 'DATE', '-',
                 'NATIONALITY', '-', 'PROFESSION', '-', 'AGE', '-', 'MONEY', '-', 'COUNTRY', '-', 'LAW', '-', 'WORK_OF_ART', '-',
                 'PERCENT', '-', 'PRODUCT', '-', 'FACILITY', '-', 'FAMILY', '-', 'NUMBER', '-', 'LOCATION', '-', 'DISTRICT', '-',
                 'STATE_OR_PROVINCE']
            else:
                entity_type = ["-", "ORG", "-",  "LOC", "-",  "TIME", "-",  "PER", "-", "MISC", "-", "NUM"]

        for sample in tqdm(data, desc="Example"):
            sents = []
            sent_map = []

            entities = sample['vertexSet']
            entity_start, entity_end = [], []
            mention_types = []
            for entity in entities:
                for mention in entity:
                    sent_id = mention["sent_id"]
                    pos = mention["pos"]
                    entity_start.append((sent_id, pos[0]))
                    entity_end.append((sent_id, pos[1] - 1))
                    mention_types.append(mention['type'])

            for i_s, sent in enumerate(sample['sents']):
                new_map = {}
                for i_t, token in enumerate(sent):
                    tokens_wordpiece = tokenizer.tokenize(token)
                    if (i_s, i_t) in entity_start:
                        t = entity_start.index((i_s, i_t))
                        if transfermers == 'bert':
                            mention_type = mention_types[t]
                            special_token_i = entity_type.index(mention_type)
                            special_token = ['[unused' + str(special_token_i) + ']']
                        else:
                            special_token = ['*']
                        tokens_wordpiece = special_token + tokens_wordpiece
                        # tokens_wordpiece = ["[unused0]"]+ tokens_wordpiece

                    if (i_s, i_t) in entity_end:
                        t = entity_end.index((i_s, i_t))
                        if transfermers == 'bert':
                            mention_type = mention_types[t]
                            special_token_i = entity_type.index(mention_type) + 50
                            special_token = ['[unused' + str(special_token_i) + ']']
                        else:
                            special_token = ['*']
                        tokens_wordpiece = tokens_wordpiece + special_token

                        # tokens_wordpiece = tokens_wordpiece + ["[unused1]"]
                        # print(tokens_wordpiece,tokenizer.convert_tokens_to_ids(tokens_wordpiece))

                    new_map[i_t] = len(sents)
                    sents.extend(tokens_wordpiece)
                new_map[i_t + 1] = len(sents)
                sent_map.append(new_map)

            if len(sents)>max_len:
                max_len=len(sents)
            if len(sents)>512:
                up512_num += 1

            train_triple = {}
            if "labels" in sample:
                for label in sample['labels']:
                    evidence = label['evidence']
                    r = int(rel2id[label['r']])

                    # h, t = sorted([label['h'], label['t']])
                    h, t = [label['h'], label['t']]

                    if (h, t) not in train_triple:
                        train_triple[(h, t)] = [
                            {'relation': r, 'evidence': evidence}]
                    else:
                        train_triple[(h, t)].append(
                            {'relation': r, 'evidence': evidence})

            entity_pos = []
            for e in entities:
                entity_pos.append([])
                mention_num = len(e)
                for m in e:
                    start = sent_map[m["sent_id"]][m["pos"][0]]
                    end = sent_map[m["sent_id"]][m["pos"][1]]
                    entity_pos[-1].append((start, end,))


            relations, hts = [], []
            # Get positive samples from dataset
            for h, t in train_triple.keys():
                relation = [0] * len(rel2id)
                for mention in train_triple[h, t]:
                    relation[mention["relation"]] = 1
                    evidence = mention["evidence"]
                relations.append(relation)
                hts.append([h, t])
                pos_samples += 1
            # print(len(relations))
            # print(entities)
            # print(sorted(list(train_triple.keys())))

            # Get negative samples from dataset
            for h in range(len(entities)):
                for t in range(len(entities)):
                    if h != t and [h, t] not in hts:
                        relation = [1] + [0] * (len(rel2id) - 1)
                        relations.append(relation)
                        hts.append([h, t])
                        neg_samples += 1

            assert len(relations) <= len(entities) * (len(entities) - 1) + 1, f'relations:{len(relations)} entities:{len(entities)}'
            assert len(relations) >= len(entities) * (len(entities) - 1), f'relations:{len(relations)} entities:{len(entities)}'

            if len(hts)==0:
                print(len(sent))
            sents = sents[:max_seq_length - 2]
            input_ids = tokenizer.convert_tokens_to_ids(sents)
            input_ids = tokenizer.build_inputs_with_special_tokens(input_ids)

            i_line += 1
            feature = {'input_ids': input_ids,
                       'entity_pos': entity_pos,
                       'labels': relations,
                       'hts': hts,
                       'title': sample['title'],
                       }
            features.append(feature)



        print("# of documents {}.".format(i_line))
        print("# of positive examples {}.".format(pos_samples))
        print("# of negative examples {}.".format(neg_samples))
        print("# {} examples len>512 and max len is {}.".format(up512_num, max_len))


        with open(file=save_file, mode='wb') as fw:
            pickle.dump(features, fw)
        print('finish reading {} and save preprocessed data to {}.'.format(file_in, save_file))


In [ ]:
# ner2id_path = './data/for_docred/ner2id.json'
# with open(ner2id_path, 'r') as f:
#     ner2id = json.load(f)

# print([
#     x
#     for ent in ner2id.keys()
#     for x in ['-', ent]
# ])

In [ ]:
# train_features = Dataset.read(train_file)

## Prepare the Model

In [ ]:
class AttentionUNet(torch.nn.Module):
    """
    UNet, down sampling & up sampling for global reasoning
    """

    def __init__(self, input_channels, class_number, **kwargs):
        super(AttentionUNet, self).__init__()

        down_channel = kwargs['down_channel'] # default = 256

        down_channel_2 = down_channel * 2
        up_channel_1 = down_channel_2 * 2
        up_channel_2 = down_channel * 2

        self.inc = InConv(input_channels, down_channel)
        self.down1 = DownLayer(down_channel, down_channel_2)
        self.down2 = DownLayer(down_channel_2, down_channel_2)

        self.up1 = UpLayer(up_channel_1, up_channel_1 // 4)
        self.up2 = UpLayer(up_channel_2, up_channel_2 // 4)
        self.outc = OutConv(up_channel_2 // 4, class_number)

    def forward(self, attention_channels):
        """
        Given multi-channel attention map, return the logits of every one mapping into 3-class
        :param attention_channels:
        :return:
        """
        # attention_channels as the shape of: batch_size x channel x width x height
        x = attention_channels
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x = self.up1(x3, x2)
        x = self.up2(x, x1)
        output = self.outc(x)
        # attn_map as the shape of: batch_size x width x height x class
        output = output.permute(0, 2, 3, 1).contiguous()
        return output


class DoubleConv(nn.Module):
    """(conv => [BN] => ReLU) * 2"""

    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
                                         nn.BatchNorm2d(out_ch),
                                         nn.ReLU(inplace=True),
                                         nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
                                         nn.BatchNorm2d(out_ch),
                                         nn.ReLU(inplace=True))

    def forward(self, x):
        x = self.double_conv(x)
        return x


class InConv(nn.Module):

    def __init__(self, in_ch, out_ch):
        super(InConv, self).__init__()
        self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class DownLayer(nn.Module):

    def __init__(self, in_ch, out_ch):
        super(DownLayer, self).__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(kernel_size=2),
            DoubleConv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.maxpool_conv(x)
        return x


class UpLayer(nn.Module):

    def __init__(self, in_ch, out_ch, bilinear=True):
        super(UpLayer, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear',
                                  align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch // 2, in_ch // 2, 2, stride=2)
        self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, (diffX // 2, diffX - diffX // 2, diffY // 2, diffY -
                        diffY // 2))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class OutConv(nn.Module):

    def __init__(self, in_ch, out_ch):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x

class DocREModel(nn.Module):
    def __init__(self, config, args, model, emb_size=768, block_size=64, num_labels=-1):
        super().__init__()
        self.config = config
        self.bert_model = model
        self.hidden_size = config.hidden_size
        self.loss_fnt = ATLoss()

        self.head_extractor = nn.Linear(1 * config.hidden_size + args.unet_out_dim, emb_size)
        self.tail_extractor = nn.Linear(1 * config.hidden_size + args.unet_out_dim, emb_size)
        # self.head_extractor = nn.Linear(1 * config.hidden_size , emb_size)
        # self.tail_extractor = nn.Linear(1 * config.hidden_size , emb_size)
        self.bilinear = nn.Linear(emb_size * block_size, config.num_labels)

        self.emb_size = emb_size
        self.block_size = block_size
        self.num_labels = num_labels

        self.bertdrop = nn.Dropout(0.6)
        self.unet_in_dim = args.unet_in_dim
        self.unet_out_dim = args.unet_in_dim
        self.liner = nn.Linear(config.hidden_size, args.unet_in_dim)
        self.min_height = args.max_height
        self.channel_type = args.channel_type
        self.segmentation_net = AttentionUNet(input_channels=args.unet_in_dim,
                                              class_number=args.unet_out_dim,
                                              down_channel=args.down_dim)


    def encode(self, input_ids, attention_mask,entity_pos):
        config = self.config
        if config.transformer_type == "bert":
            start_tokens = [config.cls_token_id]
            end_tokens = [config.sep_token_id]
        elif config.transformer_type == "roberta":
            start_tokens = [config.cls_token_id]
            end_tokens = [config.sep_token_id, config.sep_token_id]
        sequence_output, attention = process_long_input(self.bert_model, input_ids, attention_mask, start_tokens, end_tokens)
        return sequence_output, attention

    def get_hrt(self, sequence_output, attention, entity_pos, hts):
        offset = 1 if self.config.transformer_type in ["bert", "roberta"] else 0
        bs, h, _, c = attention.size()
        # ne = max([len(x) for x in entity_pos])  # 本次bs中的最大实体数

        hss, tss, rss = [], [], []
        entity_es = []
        entity_as = []
        for i in range(len(entity_pos)):
            entity_embs, entity_atts = [], []
            for entity_num, e in enumerate(entity_pos[i]):
                if len(e) > 1:
                    e_emb, e_att = [], []
                    for start, end in e:
                        if start + offset < c:
                            # In case the entity mention is truncated due to limited max seq length.
                            e_emb.append(sequence_output[i, start + offset])
                            e_att.append(attention[i, :, start + offset])
                    if len(e_emb) > 0:
                        e_emb = torch.logsumexp(torch.stack(e_emb, dim=0), dim=0)
                        e_att = torch.stack(e_att, dim=0).mean(0)
                    else:
                        e_emb = torch.zeros(self.config.hidden_size).to(sequence_output)
                        e_att = torch.zeros(h, c).to(attention)
                else:
                    start, end = e[0]
                    if start + offset < c:
                        e_emb = sequence_output[i, start + offset]
                        e_att = attention[i, :, start + offset]
                    else:
                        e_emb = torch.zeros(self.config.hidden_size).to(sequence_output)
                        e_att = torch.zeros(h, c).to(attention)
                entity_embs.append(e_emb)
                entity_atts.append(e_att)
            for _ in range(self.min_height-entity_num-1):
                entity_atts.append(e_att)

            entity_embs = torch.stack(entity_embs, dim=0)  # [n_e, d]
            entity_atts = torch.stack(entity_atts, dim=0)  # [n_e, h, seq_len]


            entity_es.append(entity_embs)
            entity_as.append(entity_atts)
            ht_i = torch.LongTensor(hts[i]).to(sequence_output.device)
            hs = torch.index_select(entity_embs, 0, ht_i[:, 0])
            ts = torch.index_select(entity_embs, 0, ht_i[:, 1])

            hss.append(hs)
            tss.append(ts)
        hss = torch.cat(hss, dim=0)
        tss = torch.cat(tss, dim=0)
        return hss, tss, entity_es, entity_as

    def get_mask(self, ents, bs, ne, run_device):
        ent_mask = torch.zeros(bs, ne, device=run_device)
        rel_mask = torch.zeros(bs, ne, ne, device=run_device)
        for _b in range(bs):
            ent_mask[_b, :len(ents[_b])] = 1
            rel_mask[_b, :len(ents[_b]), :len(ents[_b])] = 1
        return ent_mask, rel_mask


    def get_ht(self, rel_enco, hts):
        htss = []
        for i in range(len(hts)):
            ht_index = hts[i]
            for (h_index, t_index) in ht_index:
                htss.append(rel_enco[i,h_index,t_index])
        htss = torch.stack(htss,dim=0)
        return htss

    def get_channel_map(self, sequence_output, entity_as):
        # sequence_output = sequence_output.to('cpu')
        # attention = attention.to('cpu')
        bs,_,d = sequence_output.size()
        # ne = max([len(x) for x in entity_as])  # 本次bs中的最大实体数
        ne = self.min_height

        index_pair = []
        for i in range(ne):
            tmp = torch.cat((torch.ones((ne, 1), dtype=int) * i, torch.arange(0, ne).unsqueeze(1)), dim=-1)
            index_pair.append(tmp)
        index_pair = torch.stack(index_pair, dim=0).reshape(-1, 2).to(sequence_output.device)
        map_rss = []
        for b in range(bs):
            entity_atts = entity_as[b]
            h_att = torch.index_select(entity_atts, 0, index_pair[:, 0])
            t_att = torch.index_select(entity_atts, 0, index_pair[:, 1])
            ht_att = (h_att * t_att).mean(1)
            ht_att = ht_att / (ht_att.sum(1, keepdim=True) + 1e-5)
            rs = contract("ld,rl->rd", sequence_output[b], ht_att)
            map_rss.append(rs)
        map_rss = torch.cat(map_rss, dim=0).reshape(bs, ne, ne, d)
        return map_rss

    def forward(self,
                input_ids=None,
                attention_mask=None,
                labels=None,
                entity_pos=None,
                hts=None,
                instance_mask=None,
                ):

        sequence_output, attention = self.encode(input_ids, attention_mask,entity_pos)

        bs, sequen_len, d = sequence_output.shape
        # run_device = sequence_output.device.index
        run_device = sequence_output.device
        ne = max([len(x) for x in entity_pos])  # 本次bs中的最大实体数
        ent_mask, rel_mask = self.get_mask(entity_pos, bs, ne, run_device)

        # get hs, ts and entity_embs >> entity_rs
        hs, ts, entity_embs, entity_as = self.get_hrt(sequence_output, attention, entity_pos, hts)


        if self.channel_type == 'context-based':
            feature_map = self.get_channel_map(sequence_output, entity_as)
            ##print('feature_map:', feature_map.shape)
            attn_input = self.liner(feature_map).permute(0, 3, 1, 2).contiguous()

        else:
            raise Exception("channel_type must be specify correctly")


        attn_map = self.segmentation_net(attn_input)
        h_t = self.get_ht (attn_map, hts)

        hs = torch.tanh(self.head_extractor(torch.cat([hs, h_t], dim=1)))
        ts = torch.tanh(self.tail_extractor(torch.cat([ts, h_t], dim=1)))


        b1 = hs.view(-1, self.emb_size // self.block_size, self.block_size)
        b2 = ts.view(-1, self.emb_size // self.block_size, self.block_size)
        bl = (b1.unsqueeze(3) * b2.unsqueeze(2)).view(-1, self.emb_size * self.block_size)
        logits = self.bilinear(bl)


        output = (self.loss_fnt.get_label(logits, num_labels=self.num_labels))
        if labels is not None:
            labels = [torch.tensor(label) for label in labels]
            labels = torch.cat(labels, dim=0).to(logits)
            loss = self.loss_fnt(logits.float(), labels.float())
            output = (loss.to(sequence_output), output)
        return output

## Loss function

In [ ]:
def multilabel_categorical_crossentropy(y_true, y_pred):
    y_pred = (1 - 2 * y_true) * y_pred
    y_pred_neg = y_pred - y_true * 1e30
    y_pred_pos = y_pred - (1 - y_true) * 1e30
    zeros = torch.zeros_like(y_pred[..., :1])
    y_pred_neg = torch.cat([y_pred_neg, zeros],dim=-1)
    y_pred_pos = torch.cat((y_pred_pos, zeros),dim=-1)
    neg_loss = torch.logsumexp(y_pred_neg, axis=-1)
    pos_loss = torch.logsumexp(y_pred_pos, axis=-1)
    return neg_loss + pos_loss


class ATLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, logits, labels):

        loss = multilabel_categorical_crossentropy(labels,logits)
        loss = loss.mean()
        return loss

    def get_label(self, logits, num_labels=-1):
        th_logit = torch.zeros_like(logits[..., :1])
        output = torch.zeros_like(logits).to(logits)
        mask = (logits > th_logit)
        if num_labels > 0:
            top_v, _ = torch.topk(logits, num_labels, dim=1)
            top_v = top_v[:, -1]
            mask = (logits >= top_v.unsqueeze(1)) & mask
        output[mask] = 1.0
        output[:, 0] = (output[:,1:].sum(1) == 0.).to(logits)

        return output

## Preprocess the inputs

In [ ]:
def process_long_input(model, input_ids, attention_mask, start_tokens, end_tokens):
    # Split the input to 2 overlapping chunks. Now BERT can encode inputs of which the length are up to 1024.
    n, c = input_ids.size()
    start_tokens = torch.tensor(start_tokens).to(input_ids)
    end_tokens = torch.tensor(end_tokens).to(input_ids)
    len_start = start_tokens.size(0)
    len_end = end_tokens.size(0)
    if c <= 512:
        output = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_attentions=True,
        )
        sequence_output = output[0]
        attention = output[-1][-1]
    else:
        new_input_ids, new_attention_mask, num_seg = [], [], []
        seq_len = attention_mask.sum(1).cpu().numpy().astype(np.int32).tolist()
        for i, l_i in enumerate(seq_len):
            if l_i <= 512:
                new_input_ids.append(input_ids[i, :512])
                new_attention_mask.append(attention_mask[i, :512])
                num_seg.append(1)
            else:
                input_ids1 = torch.cat([input_ids[i, :512 - len_end], end_tokens], dim=-1)
                input_ids2 = torch.cat([start_tokens, input_ids[i, (l_i - 512 + len_start): l_i]], dim=-1)
                attention_mask1 = attention_mask[i, :512]
                attention_mask2 = attention_mask[i, (l_i - 512): l_i]
                new_input_ids.extend([input_ids1, input_ids2])
                new_attention_mask.extend([attention_mask1, attention_mask2])
                num_seg.append(2)
        input_ids = torch.stack(new_input_ids, dim=0)
        attention_mask = torch.stack(new_attention_mask, dim=0)
        output = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_attentions=True,
        )
        sequence_output = output[0]
        attention = output[-1][-1]
        i = 0
        new_output, new_attention = [], []
        for (n_s, l_i) in zip(num_seg, seq_len):
            if n_s == 1:
                output = F.pad(sequence_output[i], (0, 0, 0, c - 512))
                att = F.pad(attention[i], (0, c - 512, 0, c - 512))
                new_output.append(output)
                new_attention.append(att)
            elif n_s == 2:
                output1 = sequence_output[i][:512 - len_end]
                mask1 = attention_mask[i][:512 - len_end]
                att1 = attention[i][:, :512 - len_end, :512 - len_end]
                output1 = F.pad(output1, (0, 0, 0, c - 512 + len_end))
                mask1 = F.pad(mask1, (0, c - 512 + len_end))
                att1 = F.pad(att1, (0, c - 512 + len_end, 0, c - 512 + len_end))

                output2 = sequence_output[i + 1][len_start:]
                mask2 = attention_mask[i + 1][len_start:]
                att2 = attention[i + 1][:, len_start:, len_start:]
                output2 = F.pad(output2, (0, 0, l_i - 512 + len_start, c - l_i))
                mask2 = F.pad(mask2, (l_i - 512 + len_start, c - l_i))
                att2 = F.pad(att2, [l_i - 512 + len_start, c - l_i, l_i - 512 + len_start, c - l_i])
                mask = mask1 + mask2 + 1e-10
                output = (output1 + output2) / mask.unsqueeze(-1)
                att = (att1 + att2)
                att = att / (att.sum(-1, keepdim=True) + 1e-10)
                new_output.append(output)
                new_attention.append(att)
            i += n_s
        sequence_output = torch.stack(new_output, dim=0)
        attention = torch.stack(new_attention, dim=0)
    return sequence_output, attention

## Auxiliary functions

In [ ]:
def set_seed(cfg):
    random.seed(cfg.seed)
    np.random.seed(cfg.seed)
    torch.manual_seed(cfg.seed)

def collate_fn(batch):
    max_len = max([len(f["input_ids"]) for f in batch])
    input_ids = [f["input_ids"] + [0] * (max_len - len(f["input_ids"])) for f in batch]
    input_mask = [[1.0] * len(f["input_ids"]) + [0.0] * (max_len - len(f["input_ids"])) for f in batch]
    input_ids = torch.tensor(input_ids, dtype=torch.long)
    input_mask = torch.tensor(input_mask, dtype=torch.float)
    entity_pos = [f["entity_pos"] for f in batch]

    labels = [f["labels"] for f in batch]
    hts = [f["hts"] for f in batch]
    output = (input_ids, input_mask, labels, entity_pos, hts )
    return output

def to_official(args, preds, features):
    rel2id = json.load(open(f'{args.data_dir}/rel2id.json', 'r'))
    id2rel = {value: key for key, value in rel2id.items()}

    h_idx, t_idx, title = [], [], []

    for f in features:
        hts = f["hts"]
        h_idx += [ht[0] for ht in hts]
        t_idx += [ht[1] for ht in hts]
        title += [f["title"] for ht in hts]

    res = []
    # print('h_idx, preds', len(h_idx), len(preds))
    # assert len(h_idx) == len(preds)


    for i in range(preds.shape[0]):
        pred = preds[i]
        pred = np.nonzero(pred)[0].tolist()
        for p in pred:
            if p != 0:
                res.append(
                    {
                        'title': title[i],
                        'h_idx': h_idx[i],
                        't_idx': t_idx[i],
                        'r': id2rel[p],
                    }
                )
    return res

def gen_train_facts(data_file_name, truth_dir):
    fact_file_name = data_file_name[data_file_name.find("train_"):]
    fact_file_name = os.path.join(truth_dir, fact_file_name.replace(".json", ".fact"))

    if os.path.exists(fact_file_name):
        fact_in_train = set([])
        triples = json.load(open(fact_file_name))
        for x in triples:
            fact_in_train.add(tuple(x))
        return fact_in_train

    fact_in_train = set([])
    ori_data = json.load(open(data_file_name))
    for data in ori_data:
        vertexSet = data['vertexSet']
        for label in data['labels']:
            rel = label['r']
            for n1 in vertexSet[label['h']]:
                for n2 in vertexSet[label['t']]:
                    fact_in_train.add((n1['name'], n2['name'], rel))

    json.dump(list(fact_in_train), open(fact_file_name, "w"))

    return fact_in_train

def official_evaluate(tmp, path):
    '''
        Adapted from the official evaluation code
    '''
    truth_dir = os.path.join(path, 'ref')

    if not os.path.exists(truth_dir):
        os.makedirs(truth_dir)

    fact_in_train_annotated = gen_train_facts(os.path.join(path, "train_annotated.json"), truth_dir)

    # if not os.path.exists(os.path.join(path, "train_distant.json")):
    #     raise FileNotFoundError("Sorry, the file: 'train_annotated.json' is too big to upload to github, \
    #         please manually download to 'data/' from DocRED GoogleDrive https://drive.google.com/drive/folders/1c5-0YwnoJx8NS6CV2f-NoTHR__BdkNqw")
    # fact_in_train_distant = gen_train_facts(os.path.join(path, "train_distant.json"), truth_dir)

    truth = json.load(open(os.path.join(path, "dev.json")))

    std = {}
    tot_evidences = 0
    titleset = set([])

    title2vectexSet = {}

    for x in truth:
        title = x['title']
        titleset.add(title)

        vertexSet = x['vertexSet']
        title2vectexSet[title] = vertexSet

        for label in x['labels']:
            r = label['r']
            h_idx = label['h']
            t_idx = label['t']
            std[(title, r, h_idx, t_idx)] = set(label['evidence'])
            tot_evidences += len(label['evidence'])

    tot_relations = len(std)
    tmp.sort(key=lambda x: (x['title'], x['h_idx'], x['t_idx'], x['r']))
    submission_answer = [tmp[0]]
    for i in range(1, len(tmp)):
        x = tmp[i]
        y = tmp[i - 1]
        if (x['title'], x['h_idx'], x['t_idx'], x['r']) != (y['title'], y['h_idx'], y['t_idx'], y['r']):
            submission_answer.append(tmp[i])

    correct_re = 0
    correct_evidence = 0
    pred_evi = 0

    correct_in_train_annotated = 0
    correct_in_train_distant = 0
    titleset2 = set([])
    for x in submission_answer:
        title = x['title']
        h_idx = x['h_idx']
        t_idx = x['t_idx']
        r = x['r']
        titleset2.add(title)
        if title not in title2vectexSet:
            continue
        vertexSet = title2vectexSet[title]

        if 'evidence' in x:
            evi = set(x['evidence'])
        else:
            evi = set([])
        pred_evi += len(evi)

        if (title, r, h_idx, t_idx) in std:
            correct_re += 1
            stdevi = std[(title, r, h_idx, t_idx)]
            correct_evidence += len(stdevi & evi)
            in_train_annotated = in_train_distant = False
            for n1 in vertexSet[h_idx]:
                for n2 in vertexSet[t_idx]:
                    if (n1['name'], n2['name'], r) in fact_in_train_annotated:
                        in_train_annotated = True
                    # if (n1['name'], n2['name'], r) in fact_in_train_distant:
                    #     in_train_distant = True

            if in_train_annotated:
                correct_in_train_annotated += 1
            # if in_train_distant:
            #     correct_in_train_distant += 1

    re_p = 1.0 * correct_re / len(submission_answer)
    re_r = 1.0 * correct_re / tot_relations
    if re_p + re_r == 0:
        re_f1 = 0
    else:
        re_f1 = 2.0 * re_p * re_r / (re_p + re_r)

    evi_p = 1.0 * correct_evidence / pred_evi if pred_evi > 0 else 0
    evi_r = 1.0 * correct_evidence / tot_evidences
    if evi_p + evi_r == 0:
        evi_f1 = 0
    else:
        evi_f1 = 2.0 * evi_p * evi_r / (evi_p + evi_r)

    re_p_ignore_train_annotated = 1.0 * (correct_re - correct_in_train_annotated) / (len(submission_answer) - correct_in_train_annotated + 1e-5)
    re_p_ignore_train = 1.0 * (correct_re - correct_in_train_distant) / (len(submission_answer) - correct_in_train_distant + 1e-5)

    if re_p_ignore_train_annotated + re_r == 0:
        re_f1_ignore_train_annotated = 0
    else:
        re_f1_ignore_train_annotated = 2.0 * re_p_ignore_train_annotated * re_r / (re_p_ignore_train_annotated + re_r)

    if re_p_ignore_train + re_r == 0:
        re_f1_ignore_train = 0
    else:
        re_f1_ignore_train = 2.0 * re_p_ignore_train * re_r / (re_p_ignore_train + re_r)

    return re_f1, evi_f1, re_f1_ignore_train_annotated, re_f1_ignore_train, re_p, re_r

## Train the model
### Config parameters

### DeepPavlov ruBert

In [ ]:
class Config(object):
    adam_epsilon=1e-06
    bert_lr=3e-05
    channel_type='context-based'
    config_name=''
    data_dir='/content/drive/MyDrive/data/nerel_docred_ver'
    dataset='docred'
    dev_file='dev.json'
    down_dim=256
    evaluation_steps=-1
    gradient_accumulation_steps=2
    learning_rate=0.0003
    log_dir='/content/drive/MyDrive/Models/DeepKE_train/logs/train_roberta.log'
    max_grad_norm=1.0
    max_height=103
    max_seq_length=512
    # model_name_or_path='xlm-roberta-base'
    model_name_or_path='DeepPavlov/rubert-base-cased'
    num_class=50
    num_labels=29
    num_train_epochs=10
    save_path='/content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_roberta.pt'
    seed=111
    test_batch_size=2
    test_file='test.json'
    tokenizer_name=''
    train_batch_size=2
    train_file='train_annotated_filter.json'
    train_from_saved_model=''
    transformer_type='roberta'
    unet_in_dim=3
    unet_out_dim=256
    warmup_ratio=0.06
    load_path='/content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_roberta.pt'

cfg = Config()

In [ ]:
# class Config(object):
#     adam_epsilon=1e-06
#     bert_lr=3e-05
#     channel_type='context-based'
#     config_name=''
#     data_dir='/Users/yandutov-alex/study/science_work/DocRED_original/data/'
#     dataset='docred'
#     dev_file='dev.json'
#     down_dim=256
#     evaluation_steps=-1
#     gradient_accumulation_steps=2
#     learning_rate=0.0004
#     log_dir='./logs/deepke/train_roberta_docred.log'
#     max_grad_norm=1.0
#     max_height=42
#     max_seq_length=1024
#     model_name_or_path='roberta-base'
#     num_class=97
#     num_labels=4
#     num_train_epochs=30
#     save_path='./model_roberta.pt'
#     seed=111
#     test_batch_size=2
#     test_file='test.json'
#     tokenizer_name=''
#     train_batch_size=2
#     train_file='train_annotated.json'
#     train_from_saved_model=''
#     transformer_type='roberta'
#     unet_in_dim=3
#     unet_out_dim=256
#     warmup_ratio=0.06
#     load_path='./model_roberta.pt'

# cfg = Config()

### Model Training

In [ ]:
def train(args, model, train_features, dev_features, test_features):
    def logging(s, print_=True, log_=True):
        if print_:
            print(s)
        if log_ and args.log_dir != '':
            with open(args.log_dir, 'a+') as f_log:
                f_log.write(s + '\n')
    def finetune(features, optimizer, num_epoch, num_steps, model):
        cur_model = model.module if hasattr(model, 'module') else model
        # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        if torch.cuda.is_available():
            device = "cuda:0"
        # elif torch.backends.mps.is_available():
        #     device = "mps"
        else:
            device = "cpu"

        if args.train_from_saved_model != '':
            best_score = torch.load(args.train_from_saved_model)["best_f1"]
            epoch_delta = torch.load(args.train_from_saved_model)["epoch"] + 1
        else:
            epoch_delta = 0
            best_score = -1
        train_dataloader = DataLoader(features, batch_size=args.train_batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
        train_iterator = [epoch + epoch_delta for epoch in range(num_epoch)]
        total_steps = int(len(train_dataloader) * num_epoch // args.gradient_accumulation_steps)
        warmup_steps = int(total_steps * args.warmup_ratio)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
        print("Total steps: {}".format(total_steps))
        print("Warmup steps: {}".format(warmup_steps))
        global_step = 0
        log_step = 100
        total_loss = 0



        #scaler = GradScaler()
        for epoch in train_iterator:
            start_time = time.time()
            optimizer.zero_grad()

            for step, batch in tqdm(enumerate(train_dataloader)):
                model.train()

                inputs = {'input_ids': batch[0].to(device),
                          'attention_mask': batch[1].to(device),
                          'labels': batch[2],
                          'entity_pos': batch[3],
                          'hts': batch[4],
                          }
                #with autocast():
                outputs = model(**inputs)
                loss = outputs[0] / args.gradient_accumulation_steps
                total_loss += loss.item()
                #    scaler.scale(loss).backward()


                loss.backward()

                if step % args.gradient_accumulation_steps == 0:
                    #scaler.unscale_(optimizer)
                    if args.max_grad_norm > 0:
                        # torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                        torch.nn.utils.clip_grad_norm_(cur_model.parameters(), args.max_grad_norm)
                    #scaler.step(optimizer)
                    #scaler.update()
                    #scheduler.step()
                    optimizer.step()
                    scheduler.step()
                    optimizer.zero_grad()
                    global_step += 1
                    num_steps += 1
                    if global_step % log_step == 0:
                        cur_loss = total_loss / log_step
                        elapsed = time.time() - start_time
                        logging(
                            '| epoch {:2d} | step {:4d} | min/b {:5.2f} | lr {} | train loss {:5.3f}'.format(
                                epoch, global_step, elapsed / 60, scheduler.get_last_lr(), cur_loss))
                        total_loss = 0
                        start_time = time.time()

                if (step + 1) == len(train_dataloader) - 1 or (args.evaluation_steps > 0 and num_steps % args.evaluation_steps == 0 and step % args.gradient_accumulation_steps == 0):
                # if step ==0:
                    logging('-' * 89)
                    eval_start_time = time.time()
                    dev_score, dev_output = evaluate(args, model, dev_features, tag="dev")

                    logging(
                        '| epoch {:3d} | time: {:5.2f}s | dev_result:{}'.format(epoch, time.time() - eval_start_time,
                                                                                dev_output))
                    logging('-' * 89)
                    if dev_score > best_score:
                        best_score = dev_score
                        logging(
                            '| epoch {:3d} | best_f1:{}'.format(epoch, best_score))
                        if args.save_path != "":
                            torch.save({
                                'epoch': epoch,
                                'checkpoint': cur_model.state_dict(),
                                'best_f1': best_score,
                                'optimizer': optimizer.state_dict()
                            }, args.save_path
                            , _use_new_zipfile_serialization=False)
                            logging(
                                '| successfully save model at: {}'.format(args.save_path))
                            logging('-' * 89)
        return num_steps

    cur_model = model.module if hasattr(model, 'module') else model
    extract_layer = ["extractor", "bilinear"]
    bert_layer = ['bert_model']
    optimizer_grouped_parameters = [
        {"params": [p for n, p in cur_model.named_parameters() if any(nd in n for nd in bert_layer)], "lr": args.bert_lr},
        {"params": [p for n, p in cur_model.named_parameters() if any(nd in n for nd in extract_layer)], "lr": 1e-4},
        {"params": [p for n, p in cur_model.named_parameters() if not any(nd in n for nd in extract_layer + bert_layer)]},
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    if args.train_from_saved_model != '':
        optimizer.load_state_dict(torch.load(args.train_from_saved_model)["optimizer"])
        print("load saved optimizer from {}.".format(args.train_from_saved_model))


    num_steps = 0
    set_seed(args)
    model.zero_grad()
    finetune(train_features, optimizer, args.num_train_epochs, num_steps, model)

def evaluate(args, model, features, tag="dev"):
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        device = "cuda:0"
    # elif torch.backends.mps.is_available():
    #     device = "mps"
    else:
        device = "cpu"

    dataloader = DataLoader(features, batch_size=args.test_batch_size, shuffle=False, collate_fn=collate_fn, drop_last=False)
    preds = []
    total_loss = 0
    for i, batch in enumerate(dataloader):
        model.eval()

        inputs = {'input_ids': batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'labels': batch[2],
                  'entity_pos': batch[3],
                  'hts': batch[4],
                  }

        with torch.no_grad():
            output = model(**inputs)
            loss = output[0]
            pred = output[1].cpu().numpy()
            pred[np.isnan(pred)] = 0
            preds.append(pred)
            total_loss += loss.item()

    average_loss = total_loss / (i + 1)
    preds = np.concatenate(preds, axis=0).astype(np.float32)
    ans = to_official(args, preds, features)
    if len(ans) > 0:
        best_f1, _, best_f1_ign, _, re_p, re_r = official_evaluate(ans, args.data_dir)
    output = {
        tag + "_F1": best_f1 * 100,
        tag + "_F1_ign": best_f1_ign * 100,
        tag + "_re_p": re_p * 100,
        tag + "_re_r": re_r * 100,
        tag + "_average_loss": average_loss
    }
    return best_f1, output

In [ ]:
# if not os.path.exists(os.path.join(cfg.data_dir, "train_distant.json")):
#     raise FileNotFoundError("Sorry, the file: 'train_annotated.json' is too big to upload to github, \
#         please manually download to 'data/' from DocRED GoogleDrive https://drive.google.com/drive/folders/1c5-0YwnoJx8NS6CV2f-NoTHR__BdkNqw")


if torch.cuda.is_available():
    device = "cuda:0"
# elif torch.backends.mps.is_available():
#     device = "mps"
else:
    device = "cpu"


config = AutoConfig.from_pretrained(
    cfg.config_name if cfg.config_name else cfg.model_name_or_path,
    num_labels=cfg.num_class,
)
tokenizer = AutoTokenizer.from_pretrained(
    cfg.tokenizer_name if cfg.tokenizer_name else cfg.model_name_or_path,
)

Dataset = ReadDataset(cfg.dataset, tokenizer, cfg.max_seq_length, cfg.transformer_type)

train_file = os.path.join(cfg.data_dir, cfg.train_file)
dev_file = os.path.join(cfg.data_dir, cfg.dev_file)
test_file = os.path.join(cfg.data_dir, cfg.test_file)
train_features = Dataset.read(train_file)
dev_features = Dataset.read(dev_file)
test_features = Dataset.read(test_file)

model = AutoModel.from_pretrained(
    cfg.model_name_or_path,
    from_tf=bool(".ckpt" in cfg.model_name_or_path),
    config=config,
)


config.cls_token_id = tokenizer.cls_token_id
config.sep_token_id = tokenizer.sep_token_id
config.transformer_type = cfg.transformer_type

set_seed(cfg)
model = DocREModel(config, cfg,  model, num_labels=cfg.num_labels)
if cfg.train_from_saved_model != '':
    model.load_state_dict(torch.load(cfg.train_from_saved_model)["checkpoint"])
    print("load saved model from {}.".format(cfg.train_from_saved_model))

#if torch.cuda.device_count() > 1:
#    print("Let's use", torch.cuda.device_count(), "GPUs!")
#    model = torch.nn.DataParallel(model, device_ids = list(range(torch.cuda.device_count())))
model.to(device)

train(cfg, model, train_features, dev_features, test_features)

NameError: ignored

## Ru Roberta large

In [ ]:
class Config(object):
    adam_epsilon=1e-06
    bert_lr=3e-05
    channel_type='context-based'
    config_name=''
    data_dir='/content/drive/MyDrive/data/nerel_docred_ver'
    dataset='docred'
    dev_file='dev.json'
    down_dim=256
    evaluation_steps=-1
    gradient_accumulation_steps=2
    learning_rate=0.0003
    log_dir='/content/drive/MyDrive/Models/DeepKE_train/logs/train_ru_roberta.log'
    max_grad_norm=1.0
    max_height=103
    max_seq_length=512
    # model_name_or_path='xlm-roberta-base'
    model_name_or_path='ai-forever/ruRoberta-large'
    num_class=50
    num_labels=29
    num_train_epochs=20
    save_path='/content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_ru_roberta.pt'
    seed=111
    test_batch_size=2
    test_file='test.json'
    tokenizer_name=''
    train_batch_size=2
    train_file='train_annotated_filter.json'
    train_from_saved_model=''
    transformer_type='roberta'
    unet_in_dim=3
    unet_out_dim=256
    warmup_ratio=0.06
    load_path='/content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_ru_roberta.pt'

cfg = Config()

In [ ]:
# if not os.path.exists(os.path.join(cfg.data_dir, "train_distant.json")):
#     raise FileNotFoundError("Sorry, the file: 'train_annotated.json' is too big to upload to github, \
#         please manually download to 'data/' from DocRED GoogleDrive https://drive.google.com/drive/folders/1c5-0YwnoJx8NS6CV2f-NoTHR__BdkNqw")


if torch.cuda.is_available():
    device = "cuda:0"
# elif torch.backends.mps.is_available():
#     device = "mps"
else:
    device = "cpu"


config = AutoConfig.from_pretrained(
    cfg.config_name if cfg.config_name else cfg.model_name_or_path,
    num_labels=cfg.num_class,
)
tokenizer = AutoTokenizer.from_pretrained(
    cfg.tokenizer_name if cfg.tokenizer_name else cfg.model_name_or_path,
)

Dataset = ReadDataset(cfg.dataset, tokenizer, cfg.max_seq_length, cfg.transformer_type)

train_file = os.path.join(cfg.data_dir, cfg.train_file)
dev_file = os.path.join(cfg.data_dir, cfg.dev_file)
test_file = os.path.join(cfg.data_dir, cfg.test_file)
train_features = Dataset.read(train_file)
dev_features = Dataset.read(dev_file)
test_features = Dataset.read(test_file)

model = AutoModel.from_pretrained(
    cfg.model_name_or_path,
    from_tf=bool(".ckpt" in cfg.model_name_or_path),
    config=config,
)


config.cls_token_id = tokenizer.cls_token_id
config.sep_token_id = tokenizer.sep_token_id
config.transformer_type = cfg.transformer_type

set_seed(cfg)
model = DocREModel(config, cfg,  model, num_labels=cfg.num_labels)
if cfg.train_from_saved_model != '':
    model.load_state_dict(torch.load(cfg.train_from_saved_model)["checkpoint"])
    print("load saved model from {}.".format(cfg.train_from_saved_model))

#if torch.cuda.device_count() > 1:
#    print("Let's use", torch.cuda.device_count(), "GPUs!")
#    model = torch.nn.DataParallel(model, device_ids = list(range(torch.cuda.device_count())))
model.to(device)

train(cfg, model, train_features, dev_features, test_features)

load preprocessed data from /content/drive/MyDrive/data/nerel_docred_ver/train_annotated_filter_roberta_docred.pkl.
load preprocessed data from /content/drive/MyDrive/data/nerel_docred_ver/dev_roberta_docred.pkl.
load preprocessed data from /content/drive/MyDrive/data/nerel_docred_ver/test_roberta_docred.pkl.


Some weights of the model checkpoint at ai-forever/ruRoberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total steps: 3650
Warmup steps: 219


199it [02:04,  1.88it/s]

| epoch  0 | step  100 | min/b  2.07 | lr [1.3698630136986302e-05, 4.5662100456621006e-05, 0.000136986301369863] | train loss 1.701


...

363it [03:48,  1.35it/s]

-----------------------------------------------------------------------------------------
| epoch  19 | time:  8.07s | dev_result:{'dev_F1': 47.204724409448815, 'dev_F1_ign': 46.76990738756132, 'dev_re_p': 62.54564423578508, 'dev_re_r': 37.90705026873222, 'dev_average_loss': 0.1168784349364169}
-----------------------------------------------------------------------------------------
| epoch  19 | best_f1:0.47204724409448817
| successfully save model at: /content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_ru_roberta.pt


364it [04:09,  6.88s/it]

-----------------------------------------------------------------------------------------


365it [04:10,  1.46it/s]


correct parameters

In [ ]:
class Config(object):
    adam_epsilon=1e-06
    bert_lr=3e-05
    channel_type='context-based'
    config_name=''
    data_dir='/content/drive/MyDrive/data/nerel_docred_ver'
    dataset='docred'
    dev_file='dev.json'
    down_dim=256
    evaluation_steps=-1
    gradient_accumulation_steps=2
    learning_rate=0.0003
    log_dir='/content/drive/MyDrive/Models/DeepKE_train/logs/train_ru_roberta2.log'
    max_grad_norm=1.0
    max_height=103
    max_seq_length=1024
    # model_name_or_path='xlm-roberta-base'
    model_name_or_path='ai-forever/ruRoberta-large'
    num_class=50
    num_labels=2
    num_train_epochs=20
    save_path='/content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_ru_roberta2.pt'
    seed=111
    test_batch_size=2
    test_file='test.json'
    tokenizer_name=''
    train_batch_size=2
    train_file='train_annotated_filter.json'
    train_from_saved_model=''
    transformer_type='roberta'
    unet_in_dim=3
    unet_out_dim=256
    warmup_ratio=0.06
    load_path='/content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_ru_roberta2.pt'

cfg = Config()

In [ ]:
# if not os.path.exists(os.path.join(cfg.data_dir, "train_distant.json")):
#     raise FileNotFoundError("Sorry, the file: 'train_annotated.json' is too big to upload to github, \
#         please manually download to 'data/' from DocRED GoogleDrive https://drive.google.com/drive/folders/1c5-0YwnoJx8NS6CV2f-NoTHR__BdkNqw")


if torch.cuda.is_available():
    device = "cuda:0"
# elif torch.backends.mps.is_available():
#     device = "mps"
else:
    device = "cpu"


config = AutoConfig.from_pretrained(
    cfg.config_name if cfg.config_name else cfg.model_name_or_path,
    num_labels=cfg.num_class,
)
tokenizer = AutoTokenizer.from_pretrained(
    cfg.tokenizer_name if cfg.tokenizer_name else cfg.model_name_or_path,
)

Dataset = ReadDataset(cfg.dataset, tokenizer, cfg.max_seq_length, cfg.transformer_type)

train_file = os.path.join(cfg.data_dir, cfg.train_file)
dev_file = os.path.join(cfg.data_dir, cfg.dev_file)
test_file = os.path.join(cfg.data_dir, cfg.test_file)
train_features = Dataset.read(train_file)
dev_features = Dataset.read(dev_file)
test_features = Dataset.read(test_file)

model = AutoModel.from_pretrained(
    cfg.model_name_or_path,
    from_tf=bool(".ckpt" in cfg.model_name_or_path),
    config=config,
)


config.cls_token_id = tokenizer.cls_token_id
config.sep_token_id = tokenizer.sep_token_id
config.transformer_type = cfg.transformer_type

set_seed(cfg)
model = DocREModel(config, cfg,  model, num_labels=cfg.num_labels)
if cfg.train_from_saved_model != '':
    model.load_state_dict(torch.load(cfg.train_from_saved_model)["checkpoint"])
    print("load saved model from {}.".format(cfg.train_from_saved_model))

#if torch.cuda.device_count() > 1:
#    print("Let's use", torch.cuda.device_count(), "GPUs!")
#    model = torch.nn.DataParallel(model, device_ids = list(range(torch.cuda.device_count())))
model.to(device)

train(cfg, model, train_features, dev_features, test_features)

load preprocessed data from /content/drive/MyDrive/data/nerel_docred_ver/train_annotated_filter_roberta_docred.pkl.
load preprocessed data from /content/drive/MyDrive/data/nerel_docred_ver/dev_roberta_docred.pkl.
load preprocessed data from /content/drive/MyDrive/data/nerel_docred_ver/test_roberta_docred.pkl.


Some weights of the model checkpoint at ai-forever/ruRoberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total steps: 3650
Warmup steps: 219


199it [02:26,  1.54it/s]

| epoch  0 | step  100 | min/b  2.44 | lr [1.3698630136986302e-05, 4.5662100456621006e-05, 0.000136986301369863] | train loss 1.544


...

363it [04:29,  1.17it/s]

-----------------------------------------------------------------------------------------
| epoch  19 | time:  7.80s | dev_result:{'dev_F1': 53.72755305641211, 'dev_F1_ign': 53.21047837030704, 'dev_re_p': 63.02127659574468, 'dev_re_r': 46.822636737274735, 'dev_average_loss': 0.10549144724265058}
-----------------------------------------------------------------------------------------
| epoch  19 | best_f1:0.5372755305641211
| successfully save model at: /content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_ru_roberta2.pt


364it [04:49,  6.80s/it]

-----------------------------------------------------------------------------------------


365it [04:50,  1.26it/s]


## ru bert large

In [ ]:
class Config(object):
    adam_epsilon=1e-06
    bert_lr=3e-05
    channel_type='context-based'
    config_name=''
    data_dir='/content/drive/MyDrive/data/nerel_docred_ver'
    dataset='docred'
    dev_file='dev.json'
    down_dim=256
    evaluation_steps=-1
    gradient_accumulation_steps=2
    learning_rate=0.0003
    log_dir='/content/drive/MyDrive/Models/DeepKE_train/logs/train_ru_bertL.log'
    max_grad_norm=1.0
    max_height=103
    max_seq_length=512
    # model_name_or_path='xlm-roberta-base'
    model_name_or_path='ai-forever/ruBert-large'
    num_class=50
    num_labels=29
    num_train_epochs=15
    save_path='/content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_ru_bertL.pt'
    seed=111
    test_batch_size=2
    test_file='test.json'
    tokenizer_name=''
    train_batch_size=2
    train_file='train_annotated_filter.json'
    train_from_saved_model=''
    transformer_type='bert'
    unet_in_dim=3
    unet_out_dim=256
    warmup_ratio=0.06
    load_path='/content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_ru_bertL.pt'

cfg = Config()

In [ ]:
# if not os.path.exists(os.path.join(cfg.data_dir, "train_distant.json")):
#     raise FileNotFoundError("Sorry, the file: 'train_annotated.json' is too big to upload to github, \
#         please manually download to 'data/' from DocRED GoogleDrive https://drive.google.com/drive/folders/1c5-0YwnoJx8NS6CV2f-NoTHR__BdkNqw")


if torch.cuda.is_available():
    device = "cuda:0"
# elif torch.backends.mps.is_available():
#     device = "mps"
else:
    device = "cpu"


config = AutoConfig.from_pretrained(
    cfg.config_name if cfg.config_name else cfg.model_name_or_path,
    num_labels=cfg.num_class,
)
tokenizer = AutoTokenizer.from_pretrained(
    cfg.tokenizer_name if cfg.tokenizer_name else cfg.model_name_or_path,
)

Dataset = ReadDataset(cfg.dataset, tokenizer, cfg.max_seq_length, cfg.transformer_type)

train_file = os.path.join(cfg.data_dir, cfg.train_file)
dev_file = os.path.join(cfg.data_dir, cfg.dev_file)
test_file = os.path.join(cfg.data_dir, cfg.test_file)
train_features = Dataset.read(train_file)
dev_features = Dataset.read(dev_file)
test_features = Dataset.read(test_file)

model = AutoModel.from_pretrained(
    cfg.model_name_or_path,
    from_tf=bool(".ckpt" in cfg.model_name_or_path),
    config=config,
)


config.cls_token_id = tokenizer.cls_token_id
config.sep_token_id = tokenizer.sep_token_id
config.transformer_type = cfg.transformer_type

set_seed(cfg)
model = DocREModel(config, cfg,  model, num_labels=cfg.num_labels)
if cfg.train_from_saved_model != '':
    model.load_state_dict(torch.load(cfg.train_from_saved_model)["checkpoint"])
    print("load saved model from {}.".format(cfg.train_from_saved_model))

#if torch.cuda.device_count() > 1:
#    print("Let's use", torch.cuda.device_count(), "GPUs!")
#    model = torch.nn.DataParallel(model, device_ids = list(range(torch.cuda.device_count())))
model.to(device)

train(cfg, model, train_features, dev_features, test_features)

load preprocessed data from /content/drive/MyDrive/data/nerel_docred_ver/train_annotated_filter_bert_docred.pkl.
load preprocessed data from /content/drive/MyDrive/data/nerel_docred_ver/dev_bert_docred.pkl.
load preprocessed data from /content/drive/MyDrive/data/nerel_docred_ver/test_bert_docred.pkl.


Some weights of the model checkpoint at ai-forever/ruBert-large were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total steps: 2737
Warmup steps: 164


199it [02:03,  2.00it/s]

| epoch  0 | step  100 | min/b  2.06 | lr [1.8292682926829268e-05, 6.097560975609756e-05, 0.00018292682926829266] | train loss 1.736


...

363it [03:42,  1.64it/s]

-----------------------------------------------------------------------------------------
| epoch  14 | time:  6.04s | dev_result:{'dev_F1': 52.647058823529406, 'dev_F1_ign': 52.15498102099616, 'dev_re_p': 62.88976723759332, 'dev_re_r': 45.273474549478344, 'dev_average_loss': 0.09339573162984341}
-----------------------------------------------------------------------------------------
| epoch  14 | best_f1:0.526470588235294
| successfully save model at: /content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_ru_bertL.pt


364it [04:05,  7.32s/it]

-----------------------------------------------------------------------------------------


365it [04:06,  1.48it/s]


In [ ]:
from google.colab import runtime
runtime.unassign()

## ru bert base

In [ ]:
class Config(object):
    adam_epsilon=1e-06
    bert_lr=3e-05
    channel_type='context-based'
    config_name=''
    data_dir='/content/drive/MyDrive/data/nerel_docred_ver'
    dataset='docred'
    dev_file='dev.json'
    down_dim=256
    evaluation_steps=-1
    gradient_accumulation_steps=2
    learning_rate=0.0003
    log_dir='/content/drive/MyDrive/Models/DeepKE_train/logs/train_ru_bertB.log'
    max_grad_norm=1.0
    max_height=103
    max_seq_length=512
    # model_name_or_path='xlm-roberta-base'
    model_name_or_path='ai-forever/ruBert-base'
    num_class=50
    num_labels=29
    num_train_epochs=15
    save_path='/content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_ru_bertB.pt'
    seed=111
    test_batch_size=2
    test_file='test.json'
    tokenizer_name=''
    train_batch_size=2
    train_file='train_annotated_filter.json'
    train_from_saved_model=''
    transformer_type='bert'
    unet_in_dim=3
    unet_out_dim=256
    warmup_ratio=0.06
    load_path='/content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_ru_bertB.pt'

cfg = Config()

In [ ]:
# if not os.path.exists(os.path.join(cfg.data_dir, "train_distant.json")):
#     raise FileNotFoundError("Sorry, the file: 'train_annotated.json' is too big to upload to github, \
#         please manually download to 'data/' from DocRED GoogleDrive https://drive.google.com/drive/folders/1c5-0YwnoJx8NS6CV2f-NoTHR__BdkNqw")


if torch.cuda.is_available():
    device = "cuda:0"
# elif torch.backends.mps.is_available():
#     device = "mps"
else:
    device = "cpu"


config = AutoConfig.from_pretrained(
    cfg.config_name if cfg.config_name else cfg.model_name_or_path,
    num_labels=cfg.num_class,
)
tokenizer = AutoTokenizer.from_pretrained(
    cfg.tokenizer_name if cfg.tokenizer_name else cfg.model_name_or_path,
)

Dataset = ReadDataset(cfg.dataset, tokenizer, cfg.max_seq_length, cfg.transformer_type)

train_file = os.path.join(cfg.data_dir, cfg.train_file)
dev_file = os.path.join(cfg.data_dir, cfg.dev_file)
test_file = os.path.join(cfg.data_dir, cfg.test_file)
train_features = Dataset.read(train_file)
dev_features = Dataset.read(dev_file)
test_features = Dataset.read(test_file)

model = AutoModel.from_pretrained(
    cfg.model_name_or_path,
    from_tf=bool(".ckpt" in cfg.model_name_or_path),
    config=config,
)


config.cls_token_id = tokenizer.cls_token_id
config.sep_token_id = tokenizer.sep_token_id
config.transformer_type = cfg.transformer_type

set_seed(cfg)
model = DocREModel(config, cfg,  model, num_labels=cfg.num_labels)
if cfg.train_from_saved_model != '':
    model.load_state_dict(torch.load(cfg.train_from_saved_model)["checkpoint"])
    print("load saved model from {}.".format(cfg.train_from_saved_model))

#if torch.cuda.device_count() > 1:
#    print("Let's use", torch.cuda.device_count(), "GPUs!")
#    model = torch.nn.DataParallel(model, device_ids = list(range(torch.cuda.device_count())))
model.to(device)

train(cfg, model, train_features, dev_features, test_features)

load preprocessed data from /content/drive/MyDrive/data/nerel_docred_ver/train_annotated_filter_bert_docred.pkl.
load preprocessed data from /content/drive/MyDrive/data/nerel_docred_ver/dev_bert_docred.pkl.
load preprocessed data from /content/drive/MyDrive/data/nerel_docred_ver/test_bert_docred.pkl.


Some weights of the model checkpoint at ai-forever/ruBert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total steps: 2737
Warmup steps: 164


200it [01:41,  3.03it/s]

| epoch  0 | step  100 | min/b  1.70 | lr [1.8292682926829268e-05, 6.097560975609756e-05, 0.00018292682926829266] | train loss 1.729


...

363it [03:02,  2.02it/s]

-----------------------------------------------------------------------------------------
| epoch  14 | time:  4.56s | dev_result:{'dev_F1': 54.39970171513796, 'dev_F1_ign': 53.926676781644325, 'dev_re_p': 66.28805088596093, 'dev_re_r': 46.12709453050901, 'dev_average_loss': 0.09713644264860356}
-----------------------------------------------------------------------------------------
| epoch  14 | best_f1:0.5439970171513796
| successfully save model at: /content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_ru_bertB.pt


364it [03:13,  3.49s/it]

-----------------------------------------------------------------------------------------


365it [03:14,  1.88it/s]


### xlm roberta

In [ ]:
class Config(object):
    adam_epsilon=1e-06
    bert_lr=3e-05
    channel_type='context-based'
    config_name=''
    data_dir='/content/drive/MyDrive/data/nerel_docred_ver'
    dataset='docred'
    dev_file='dev.json'
    down_dim=256
    evaluation_steps=-1
    gradient_accumulation_steps=2
    learning_rate=0.0004
    log_dir='/content/drive/MyDrive/Models/DeepKE_train/logs/model_xlm_roberta.log'
    max_grad_norm=1.0
    max_height=103
    max_seq_length=1024
    # model_name_or_path='xlm-roberta-base'
    model_name_or_path='xlm-roberta-large'
    num_class=50
    num_labels=29
    num_train_epochs=20
    save_path='/content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_xlm_roberta.pt'
    seed=111
    test_batch_size=2
    test_file='test.json'
    tokenizer_name=''
    train_batch_size=2
    train_file='train_annotated_filter.json'
    train_from_saved_model=''
    transformer_type='roberta'
    unet_in_dim=3
    unet_out_dim=256
    warmup_ratio=0.06
    load_path='/content/drive/MyDrive/Models/DeepKE_train/checkpoints/model_xlm_roberta.pt'

cfg = Config()

In [ ]:
# if not os.path.exists(os.path.join(cfg.data_dir, "train_distant.json")):
#     raise FileNotFoundError("Sorry, the file: 'train_annotated.json' is too big to upload to github, \
#         please manually download to 'data/' from DocRED GoogleDrive https://drive.google.com/drive/folders/1c5-0YwnoJx8NS6CV2f-NoTHR__BdkNqw")


if torch.cuda.is_available():
    device = "cuda:0"
# elif torch.backends.mps.is_available():
#     device = "mps"
else:
    device = "cpu"


config = AutoConfig.from_pretrained(
    cfg.config_name if cfg.config_name else cfg.model_name_or_path,
    num_labels=cfg.num_class,
)
tokenizer = AutoTokenizer.from_pretrained(
    cfg.tokenizer_name if cfg.tokenizer_name else cfg.model_name_or_path,
)

Dataset = ReadDataset(cfg.dataset, tokenizer, cfg.max_seq_length, cfg.transformer_type)

train_file = os.path.join(cfg.data_dir, cfg.train_file)
dev_file = os.path.join(cfg.data_dir, cfg.dev_file)
test_file = os.path.join(cfg.data_dir, cfg.test_file)
train_features = Dataset.read(train_file)
dev_features = Dataset.read(dev_file)
test_features = Dataset.read(test_file)

model = AutoModel.from_pretrained(
    cfg.model_name_or_path,
    from_tf=bool(".ckpt" in cfg.model_name_or_path),
    config=config,
)


config.cls_token_id = tokenizer.cls_token_id
config.sep_token_id = tokenizer.sep_token_id
config.transformer_type = cfg.transformer_type

set_seed(cfg)
model = DocREModel(config, cfg,  model, num_labels=cfg.num_labels)
if cfg.train_from_saved_model != '':
    model.load_state_dict(torch.load(cfg.train_from_saved_model)["checkpoint"])
    print("load saved model from {}.".format(cfg.train_from_saved_model))

#if torch.cuda.device_count() > 1:
#    print("Let's use", torch.cuda.device_count(), "GPUs!")
#    model = torch.nn.DataParallel(model, device_ids = list(range(torch.cuda.device_count())))
model.to(device)

train(cfg, model, train_features, dev_features, test_features)

 of documents 730.
 of positive examples 23353.
 of negative examples 1169361.
 352 examples len>512 and max len is 1060.
finish reading /content/drive/MyDrive/data/nerel_docred_ver/train_annotated_filter.json and save preprocessed data to /content/drive/MyDrive/data/nerel_docred_ver/train_annotated_filter_roberta_docred.pkl.
Example: 100%|██████████| 94/94 [00:06<00:00, 13.79it/s]
 of documents 94.
 of positive examples 3143.
 of negative examples 170057.
 44 examples len>512 and max len is 1295.
finish reading /content/drive/MyDrive/data/nerel_docred_ver/dev.json and save preprocessed data to /content/drive/MyDrive/data/nerel_docred_ver/dev_roberta_docred.pkl.
Example: 100%|██████████| 93/93 [00:08<00:00, 11.22it/s]
 of documents 93.
 of positive examples 3259.
 of negative examples 186208.
 52 examples len>512 and max len is 3101.

In [ ]:
from google.colab import runtime
runtime.unassign()

### Model Prediction

In [ ]:
def report(args, model, features):

    if torch.cuda.is_available():
        device = "cuda:0"
    # elif torch.backends.mps.is_available():
    #     device = "mps"
    else:
        device = "cpu"

    dataloader = DataLoader(features, batch_size=args.test_batch_size, shuffle=False, collate_fn=collate_fn, drop_last=False)
    preds = []
    for batch in dataloader:
        model.eval()

        inputs = {'input_ids': batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'entity_pos': batch[3],
                  'hts': batch[4],
                  }

        with torch.no_grad():
            pred = model(**inputs)
            pred = pred.cpu().numpy()
            pred[np.isnan(pred)] = 0
            preds.append(pred)

    preds = np.concatenate(preds, axis=0).astype(np.float32)
    preds = to_official(args, preds, features)
    return preds

model.load_state_dict(torch.load(cfg.load_path)['checkpoint'])
T_features = test_features  # Testing on the test set
#T_score, T_output = evaluate(cfg, model, T_features, tag="test")
pred = report(cfg, model, T_features)
with open("./result.json", "w") as fh:
    json.dump(pred, fh)